## Imports

In [10]:
import os
import data_loading as dl
import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Part 1: Loading the Experiment(s) Data
Based on the data retrieved from `data_analysis.ipynb`, get that data into a format capable of being used by a Machine Learning model.

<span style="color: orange;">**Future Experiment:** Generating multiple months or simply estimating a year in advance.</span>

In [11]:
MODEL_DATA_DIRECTORY = '../model_data/wheat_model_data/'
N_TREND_SAMPLES = 3 # Number of price samples for "history" trend (training data)
N_MONTHS_AHEAD = 1 # Number of months ahead to predict
TEST_SIZE = 0.2
OUTPUT_COLUMN_NAME = 'Price'

# Get the number of features for this model (given the model_data chosen)
sample_file_path = os.path.join(MODEL_DATA_DIRECTORY, os.listdir(MODEL_DATA_DIRECTORY)[0])
sample_columns = pd.read_csv(sample_file_path).columns[1:] # Skip 'Unnamed: 0' column
num_input_samples = len(sample_columns[2:-1]) + N_TREND_SAMPLES + 1 # +1 for 'n_previous_prices' later
print("Num Features: ", num_input_samples)

Num Features:  9


In [12]:
model_data = dl.get_model_data_from_directory(MODEL_DATA_DIRECTORY, sample_columns, N_TREND_SAMPLES)
model_data

c:\Git\Predicting-Commodity-Food-Pricing\pytorch_conversion\data_loading.py:176: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_data = pd.concat([model_data, data_to_concat], ignore_index=True).drop(columns=['Unnamed: 0'])


,Date,Price,Proteus2,Food Price Index,Cereals Price Index,Wheat Futures,Harvest,Sentiment,n_previous_prices
0,2003-02-01,0.353945,0.368133,0.405947,0.370020,0.357385,0.383057,1.0,"[0.307030630305142, 0.2854220135885636, 0.3188..."
1,2003-03-01,0.368640,0.416001,0.374578,0.347719,0.327074,0.376625,1.0,"[0.2854220135885636, 0.3188965093387809, 0.353..."
2,2003-04-01,0.343996,0.468290,0.366889,0.343675,0.306746,0.369808,1.0,"[0.3188965093387809, 0.3539453436152275, 0.368..."
3,2003-05-01,0.401215,0.518191,0.377723,0.366430,0.333327,0.363512,1.0,"[0.3539453436152275, 0.3686397613884553, 0.343..."
4,2003-06-01,0.407073,0.569035,0.363607,0.356144,0.292171,0.357312,1.0,"[0.3686397613884553, 0.3439957116460621, 0.401..."
...,...,...,...,...,...,...,...,...,...
882,2016-08-01,0.348200,0.678242,0.588079,0.458857,0.333606,0.756849,1.0,"[0.3168927164383402, 0.3456880352521089, 0.331..."
883,2016-09-01,0.364530,0.656036,0.623093,0.439521,0.360232,0.756083,1.0,"[0.3456880352521089, 0.3311239570156358, 0.348..."
884,2016-10-01,0.393514,0.634037,0.639507,0.463615,0.388047,0.755328,1.0,"[0.3311239570156358, 0.3482003383651588, 0.364..."
885,2016-11-01,0.404325,0.610774,0.654949,0.473455,0.401033,0.754534,1.0,"[0.3482003383651588, 0.3645302430734246, 0.393..."


In [13]:
inputs_and_outputs = model_data.apply(dl.divide_inputs_and_outputs, axis=1)
inputs = inputs_and_outputs['inputs'].tolist(); outputs = inputs_and_outputs['output'].tolist()

print('inputs shape:', np.array(inputs).shape)
print('input:', inputs[0])

x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=TEST_SIZE, shuffle=False) 

print('x_train shape:', np.array(x_train).shape)

x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor, scaler = dl.format_for_ML_usage_torch(inputs, outputs, num_input_samples, num_features=9)

print('x_train_tensor shape:', x_train_tensor.shape)
print('x_test_tensor shape:', x_test_tensor.shape)
print('y_train_tensor shape:', y_train_tensor.shape)
print('y_test_tensor shape:', y_test_tensor.shape)

x_train_tensor = torch.tensor(x_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
test_data = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)


inputs shape: (887, 9)
input: [0.3681326395264126, 0.4059467085633423, 0.3700199350892781, 0.3573849110164853, 0.383056505368272, 1.0, 0.307030630305142, 0.2854220135885636, 0.3188965093387809]
x_train shape: (709, 9)


TypeError: format_for_ML_usage_torch() missing 1 required positional argument: 'num_features'

## Part 2: Building Complex ML Models

### Model 1: Simple, 1-Layer Transformer

In [ ]:
# Model Architecture
HEAD_SIZE = 64
NUM_HEADS = 8
FF_DIM = 16

# ML Optimizer
LEARNING_RATE = 1e-3
CLIP_VALUE = 0.5 # Gradient Clipping (https://neptune.ai/blog/understanding-gradient-clipping-and-how-it-can-fix-exploding-gradients-problem)
DROP_OUT_RATE = 0.2

d_model = NUM_HEADS
length = x_train_tensor.shape[1]  

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # Create a long enough positional encoding
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        # Register as a buffer that is not a model parameter
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x is [batch_size, seq_len, feature_size]
        # Adjust positional encoding to have the same size as the input
        pe = self.pe[:x.size(1), :]  # Shape: [seq_len, d_model]
        pe = pe.squeeze(1)  # Remove the singleton dimension
        # Ensure pe is expanded to match the batch size of x
        pe = pe.unsqueeze(0).repeat(x.size(0), 1, 1)  # Shape: [batch_size, seq_len, d_model]
        # The add operation below should now be valid
        return x + pe

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, head_size, num_heads, ff_dim, dropout=0):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout)
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=ff_dim, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=ff_dim, out_channels=d_model, kernel_size=1)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value):
        # Note: MultiheadAttention expects input of shape (L, N, E) where L is the sequence length, N is the batch size, and E is the embedding dimension.
        attn_output, _ = self.attention(query, key, value)
        out1 = self.layernorm1(query + attn_output)
        
        # Conv1D layers expect input of shape (N, C, L), hence we permute
        out1_permuted = out1.permute(1, 2, 0)
        ff_output = F.relu(self.conv1(out1_permuted))
        ff_output = self.conv2(ff_output)
        
        # Permute back to match the MultiheadAttention output shape
        ff_output = ff_output.permute(2, 0, 1)
        out2 = self.layernorm2(out1 + self.dropout(ff_output))
        
        return out2


In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, num_input_samples, d_model, head_size, num_heads, ff_dim, dropout=0, num_transformers=10):
        super(TransformerModel, self).__init__()  # Corrected super() call
        self.d_model = d_model
        self.input_projection = nn.Linear(9, d_model)
        self.pos_encoding = PositionalEncoding(d_model, num_input_samples)
        self.transformers = nn.ModuleList([
            TransformerBlock(d_model, head_size, num_heads, ff_dim, dropout) 
            for _ in range(num_transformers)
        ])
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)
        self.output_layer = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_projection(x)  # Projects input to d_model dimensions
        x = x + self.pos_encoding(x)  # Apply positional encoding

        for transformer in self.transformers:
            x = transformer(x, x, x)  # Process through transformer blocks

        x = x.mean(dim=1, keepdim=True)  # Aggregate features
        x = self.output_layer(x)  # Apply the output layer to get the final prediction
        
        return x

In [ ]:
# Define the model, loss function, and optimizer
model = TransformerModel(num_input_samples, d_model=d_model, head_size=HEAD_SIZE, num_heads=NUM_HEADS, ff_dim=FF_DIM, dropout=DROP_OUT_RATE)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Part 3: Training the Model
This section is specifically for training the model built in the previous section. Some contants (`NUM_EPOCHS`, `BATCH_SIZE`) are provided and should be the only required parameters to adjust for this section of the experiment. 

A plot of the loss throughout the training process is provided for easy understanding about if the model is overfitting or underfitting. For a review of these concepts, see [this article](https://www.analyticsfordecisions.com/overfitting-and-underfitting/#:~:text=Overfitting%20happens%20when%20the%20model%20is%20too%20complex,poor%20performance%20on%20both%20training%20and%20test%20datasets.).
<br/><br/>
**Potential Future Parameters**
* **Regularization:** L1, L2, Dropout; helps prevent overfitting

In [ ]:
# def learning_rate_scheduler(epoch, lr):
#     """
#     Learning rate scheduler
#     :param int epoch: current epoch
#     :param float lr: current learning rate
#     """
#     if epoch < 15:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)
    
# callback = LearningRateScheduler(learning_rate_scheduler)

In [ ]:
NUM_EPOCHS = 20
BATCH_SIZE = 3 # Fiscal Quarters

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

print(train_loader.dataset.tensors[0].shape)

In [ ]:
def train(model, data_loader, optimizer, criterion):
    model.train()  # Set the model to training mode
    total_loss = 0.
    for batch, (input, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(input)
        output = output.reshape(-1)
        target = target.reshape(-1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

# Example data loading and training loop
# Assuming data_loader is defined and loads your dataset
train_loss = []
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train_loss.append(train(model, train_loader, optimizer, criterion))
    print(f'| epoch {epoch:3d} | train_loss {train_loss[-1]:5.2f} |')


In [ ]:
# Plot the loss over time
plt.plot(train_loss)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

## Part 4: Examining Results
In this section, we're plotting the model's predictions versus the actual price point for the commodity in question. However, one plot focuses specifically on the testing data only (this is a better plot to see how well the model is performing/generalizing) and the other focuses on the entire dataset (this is a better plot to see if the model is correlating to the provided dataset at all).

Therefore, when **evaluating the performance** of the model, **the first plot should be used.** 

In [ ]:
# Ensure the model is in evaluation mode
from sklearn.metrics import mean_absolute_error


model.eval()

# Turn off gradients for validation, saves memory and computations
with torch.no_grad():
    predictions = model(x_test_tensor)
    
predictions = predictions.reshape(-1)
plt.plot(y_test, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Model Predictions (Testing Data Only)')
plt.ylabel('Price')
plt.xlabel('Months')
plt.legend()
plt.show()

# Calculate the Mean Absolute Error
final_mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error (FINAL ACCURACY METRIC):', final_mae)

In [ ]:
# Save the model to saved_models
model.save(f'saved_models/wheat_price_transformer_model_{final_mae}.h5')